In [ ]:
# Fetching Project details and data processing part in Colab Jupyter Notebook directly from BigQuery
# It always beeter to analyze data in Notebook instance
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'caramel-park-375012' # Project ID inserted based on the query results selected to explore
location = 'us-central1' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)  #bigquery instance
data_table.enable_dataframe_formatter()
auth.authenticate_user()  #Authenticate the user before using

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=caramel-park-375012:us-central1:bquxjob_73fbf4a_185fde7fe44)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job from BigQuery

job = client.get_job('bquxjob_36729913_185fdef751e') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT* FROM `caramel-park-375012.ulb_fraud_detection.fraud`




# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_36729913_185fdef751e') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results.head(8)

In [ ]:
results['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: Int64

In [ ]:
#Splitting the dataset into train, test and validate parts in ratio of 80 : 10 : 10 respectively.
# It creates a transaction Id (for Unique Id) and  split column for split part String as (TRAIN or VALIDATE or TEST)
# Please go through FARM_FINGERPRINT() used for splitting data exactly same everytime anyone call this query. 
#It is very useful for dataset split reproducibility in order to compare results.
dataset = 'ulb_fraud_detection'
bq_table = 'fraud'
query = f"""
CREATE TABLE IF NOT EXISTS `{project}.{dataset}.{bq_table}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{project}.{dataset}.{bq_table}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = client.query(query = query)
job.result()

In [ ]:
(job.ended - job.started).total_seconds()

7.497

In [ ]:
# Verifying the Split percentage as 80:10:10
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{project}.{dataset}.{bq_table}_prepped`
GROUP BY splits
"""
client.query(query = query).to_dataframe()

## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
# describe the basic characteristics of dataset
results.describe()